# CS Project 175 Phase 2

In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np

from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model 
from sklearn import metrics 
from sklearn import preprocessing

import seaborn

## Building the Pipeline

In [ ]:
RANDOM_STATE = 42

def gather_data():
    data = pd.read_csv('./data/data.csv')
    return data['lyrics'], data['genre']

def vectorize_labels(labels, classes=None):
    '''
    Vectorizes the labels.
    Returns as (indexes, labels)
    '''
    if classes is None:
        return pd.factorize(labels)
    return pd.Categorical(labels, categories=classes).codes, classes


# PHASE 1 START ----------------------------------------------------------------
def features_bow(data):
    vectorizer = CountVectorizer(stop_words='english', min_df=0.01, ngram_range=(1, 2))
    text = data.to_list()
    X = vectorizer.fit_transform(text)
    return X, vectorizer

def train_model_logistic(X, Y):
    classifier = linear_model.LogisticRegression(penalty='l2', multi_class='multinomial', class_weight='balanced', random_state=RANDOM_STATE, fit_intercept=True)
    classifier.fit(X, Y)
    return classifier

def evaluate_model_sklearn(model, X_train, Y_train, X_test, Y_test):
    train_accuracy = model.score(X_train, Y_train)
    print('\nTraining:')
    print(' accuracy:',format( 100*train_accuracy , '.2f') ) 

    # Compute and print accuracy on the test data
    print('\nTesting: ')
    test_accuracy = model.score(X_test, Y_test)
    print(' accuracy:', format( 100*test_accuracy , '.2f') )

    # Compute and print AUC on the test data
    class_probabilities = model.predict_proba(X_test)
    test_auc_score = metrics.roc_auc_score(Y_test, class_probabilities, multi_class='ovo')
    print(' AUC value:', format( 100*test_auc_score , '.2f') )

def sample_incorrect_predictions(predictions, probabilities, actuals, classes, titles, lyrics):
    np.random.seed(RANDOM_STATE)
    NUM_EXAMPLES = 10
    for _ in range(NUM_EXAMPLES):
        i = np.random.choice(np.where(predictions != actuals)[0])
        print("Song Title:", titles[i])
        print('Predicted:', classes[predictions[i]], 'Actual:', classes[actuals[i]])
        print('Probability:', probabilities[i][predictions[i]])
        print("Lyrics: ")
        print('"' + lyrics[i][:100] + '..."')
        print()

# PHASE 1 END ------------------------------------------------------------------

# PHASE 2 START ----------------------------------------------------------------
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def train_model_mlp(X, Y, hidden_layer_sizes=(100,), max_iter=200):
    classifier = MLPClassifier(solver='lbfgs', hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, random_state=RANDOM_STATE)
    classifier.fit(X, Y)
    return classifier
    
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import os

def features_word2vec(data):
    word_lists = data.str.split()
    if not os.path.exists("word2vec.wordvectors"):
        print("Did not find pre-trained embeddings, training now...")
        model = Word2Vec(sentences=word_lists, vector_size=200, window=5, min_count=1, workers=4)
        word_vectors = model.wv
        word_vectors.save("word2vec.wordvectors")
    else:
        print("Found pre-trained embeddings, loading now...")
        word_vectors = KeyedVectors.load("word2vec.wordvectors", mmap='r')
    print("Vectorizing features...")
    res = []
    i = 0
    for word_list in word_lists:
        sub = []
        for word in word_list:
            if word in word_vectors:
                sub.append(word_vectors[word])
        res.append(sub)
        if i % 1000 == 0:
            print(i, "/", len(word_lists))
        i += 1
    return res

def clean_vectors(X, Y):
    # remove inputs with too few vectors
    small_inputs = [i for i, x in enumerate(X) if len(x) < 10]
    small_inputs_set = set(small_inputs)
    X = [v for i, v in enumerate(X) if i not in small_inputs_set]
    Y = np.delete(Y, small_inputs)
    return X, Y

def train_model_rnn(X, Y):
    pass

def evaluate_model_rnn(model, X_train, Y_train, X_test, Y_test):
    pass

# PHASE 2 END ------------------------------------------------------------------

## Executing the Pipeline

### MLP on BOW

Let's start with the MLP classifier on BOW, then we'll try RNN on embeddings

In [ ]:
# Phase 2 pipeline
inputs, labels = gather_data()

In [ ]:
# convert to classes
Y, classes = vectorize_labels(labels)

In [ ]:
# Let's try MLP on BOW first
X_bow, vectorizer = features_bow(inputs)
X_bow_train, X_bow_test, Y_bow_train, Y_bow_test = train_test_split(X_bow, Y, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
model_nn = train_model_mlp(X_bow_train, Y_bow_train)

In [ ]:
# evaluate MLP on bow
evaluate_model_sklearn(model_nn, X_bow_train, Y_bow_train, X_bow_test, Y_bow_test)

### DNN on BOW

In [11]:
# 60.18 (100, 50, 50, 50, 50) 
# 60.21 (100, 100, 100, 100, 100) 3x longer to train
# 79.61 60.32 (200, 200) 21m 49.5s
# 76.09 60.66 (200, 100, 100, 100) 69m 21.4s
# 84.28 60.76 (500, 100, 100, 100) 183m 23.2s
model_dnn = train_model_mlp(X_bow_train, Y_bow_train, hidden_layer_sizes=(500, 100, 100, 100))
evaluate_model_sklearn(model_dnn, X_bow_train, Y_bow_train, X_bow_test, Y_bow_test)

c:\Users\luker\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



Training:
 accuracy: 84.28

Testing: 
 accuracy: 60.76
 AUC value: 81.35


In [ ]:
# Grid Search
from sklearn.model_selection import GridSearchCV

parameters = {
    'hidden_layer_sizes':[(500, 500), (500, 250, 250)]
}
mlp = MLPClassifier(solver='lbfgs', random_state=RANDOM_STATE)
clf = GridSearchCV(mlp, parameters, n_jobs=4)
clf.fit(X_bow, Y)

In [ ]:
print(clf.best_params_)
print(clf.best_score_)

### RNN on Embeddings

Now we're gonna try RNN on embeddings

In [ ]:
# takes a long time to run!
X_raw = features_word2vec(inputs)

In [ ]:
X_vec, Y_vec = clean_vectors(X_raw, Y)

In [ ]:
# split into training and test
X_vec_train, X_vec_test, Y_vec_train, Y_vec_test = train_test_split(X_vec, Y_vec, test_size = 0.2, random_state=RANDOM_STATE)

In [ ]:
# train model
model_rnn = train_model_rnn(X_vec_train, Y_vec_train)

## Evaluating the Model

In [ ]:
evaluate_model_rnn(model_rnn, X_vec_train, Y_vec_train, X_vec_test, Y_vec_test)

# Comparing the Models

Comparing the results of Logistic Regression on BOW, Neural Network on BOW, Deep Neural Neural Network on BOW, and RNN on Word Embeddings: